# Multi-Feature Banking Adoption and Customer Churn Prediction
## Step 2: Data Pre-processing

**Goal:** Prepare a clean dataset ready for exploratory data analysis and modeling.

Based on the issues identified in Step 1, we will perform the following data cleaning tasks:

**Tasks to Complete:**
1. Handle missing values in Surname and Age
2. Remove duplicate Tenure column
3. Convert Balance and EstimatedSalary from text to numeric
4. Convert HasCrCard and IsActiveMember from Yes/No to binary (1/0)
5. Standardize Geography values (FRA/France/French → consistent format)
6. Verify all data type conversions

In [1]:
#Importing pandas libraries

import pandas as pd
import numpy as np

#Display setting for pandas
pd.set_option('display.max_columns', None)              #show all columns
pd.set_option('display.float_format','{:.2f}'.format)    #format all decimals to 2 places

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
#Load the dataset
df = pd.read_csv('../data/Bank_Churn_Messy.csv', encoding='latin-1')

# Display dataset dimensions
print("Dataset loaded successfully!")
print("="*70)
print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

Dataset loaded successfully!
Dataset Shape: 10,001 rows × 14 columns


### 2.1 Handling Missing Values

We identified 6 missing values:
- Surname: 3 missing
- Age: 3 missing

**Strategy:**
- Since missing values represent only 0.03% of the data (3 out of 10,001 rows)
- Age is an important feature for churn analysis
- We will **`remove the 3 rows`** with missing values rather than imputing, as:
  - The loss of 3 rows is negligible
  - Imputing age could introduce bias
  - Surname is not useful for analysis anyway

First, we need to verify these are the same 3 rows with both missing values.

In [3]:
# Identify rows with missing values
print("Rows with missing values:")

# Find rows where either Surname or Age is missing
missing_rows = df[df['Surname'].isnull() | df['Age'].isnull()]
print(f"Total rows with missing values: {len(missing_rows)}")
print("\nDetails of rows with missing data:")
print(missing_rows[['CustomerId', 'Surname', 'Age', 'Geography', 'Gender', 'Exited']])

Rows with missing values:
Total rows with missing values: 3

Details of rows with missing data:
      CustomerId Surname  Age Geography  Gender  Exited
28      15728693     NaN  NaN   Germany  Female       0
121     15580203     NaN  NaN     Spain    Male       0
9389    15756954     NaN  NaN    France  Female       0


In [4]:
# Remove rows with missing values
original_size = len(df)                         # Store original dataset size
df = df.dropna(subset=['Surname', 'Age'])


new_size = len(df)                              # Store new size

# Displaying results
print("Missing Values Removal:")

print(f"Original dataset size: {original_size:,} rows")
print(f"Rows removed: {original_size - new_size}")
print(f"New dataset size: {new_size:,} rows")
print("="*70)

# Verifying no missing values remain
print(f"\nRemaining missing values:")
print(df.isnull().sum().sum())

Missing Values Removal:
Original dataset size: 10,001 rows
Rows removed: 3
New dataset size: 9,998 rows

Remaining missing values:
0


### 2.2 Removing Duplicate Column

The dataset contains two identical Tenure columns:
- Tenure (column 6)
- Tenure.1 (column 10)

Statistical analysis confirmed they have identical values. We will remove Tenure.1 (keeping the original Tenure column)

In [5]:
# Remove duplicate Tenure column
print("Removing Duplicate Column:")

print(f"Columns before removal: {len(df.columns)}")

df = df.drop('Tenure.1', axis=1)                        # Remove Tenure.1

print(f"Columns after removal: {len(df.columns)}")
print("\nTenure.1 column removed successfully")

Removing Duplicate Column:
Columns before removal: 14
Columns after removal: 13

Tenure.1 column removed successfully


### 2.3 Converting Balance and EstimatedSalary to Numeric

**Issue:** Balance and EstimatedSalary are stored as text (object) due to special characters (currency symbols).

**Strategy:**
- Remove all non-numeric characters (keeping only digits and decimal points)
- Convert to float type
- Verify conversion success

In [6]:
# Clean and convert Balance to numeric
print("Converting Balance to numeric:")

# Use regex pattern to remove non-numeric characters
# r'[^\d.]' means: remove anything that is NOT a digit (\d) or decimal point (.)
# The ^ inside [] means "NOT" - so [^\d.] = "not digit or dot"
# This removes currency symbols like £, €, $ while keeping numbers intact
df['Balance'] = df['Balance'].str.replace(r'[^\d.]', '', regex=True).astype(float)
print("Balance converted to numeric")

# Apply same regex cleaning to EstimatedSalary
df['EstimatedSalary'] = df['EstimatedSalary'].str.replace(r'[^\d.]', '', regex=True).astype(float)
print("EstimatedSalary converted to numeric")



# Verifying successful conversion
print("\nVerification:")
print(f"Balance data type: {df['Balance'].dtype}")
print(f"EstimatedSalary data type: {df['EstimatedSalary'].dtype}")

Converting Balance to numeric:
Balance converted to numeric
EstimatedSalary converted to numeric

Verification:
Balance data type: float64
EstimatedSalary data type: float64


### 2.4 Converting Binary Columns to Numeric

**Issue:** HasCrCard and IsActiveMember are stored as "Yes"/"No" text instead of binary values.

**Why convert to 1/0:**
- Machine learning models require numeric input
- Binary format (1/0) is standard for Yes/No data
- Enables mathematical operations and correlations

**Conversion:**
- "Yes" → 1
- "No" → 0

In [7]:
# Converting Yes → 1, No → 0
df['HasCrCard'] = df['HasCrCard'].map({'Yes': 1, 'No': 0})
df['IsActiveMember'] = df['IsActiveMember'].map({'Yes': 1, 'No': 0})

print("HasCrCard converted to binary (Yes → 1, No → 0)")
print("IsActiveMember converted to binary (Yes → 1, No → 0)")

print("="*70)

# Verifying conversion successful
print("\nVerification:")
print(f"HasCrCard - Type: {df['HasCrCard'].dtype}, Values: {df['HasCrCard'].unique()}")
print(f"IsActiveMember - Type: {df['IsActiveMember'].dtype}, Values: {df['IsActiveMember'].unique()}")


HasCrCard converted to binary (Yes → 1, No → 0)
IsActiveMember converted to binary (Yes → 1, No → 0)

Verification:
HasCrCard - Type: int64, Values: [1 0]
IsActiveMember - Type: int64, Values: [1 0]


### 2.5 Standardizing Geography Values

**Issue:** Geography column has inconsistent country formats:
- "FRA", "France", "French" all represent France
- This creates problems for grouping and analysis

**Strategy:**
1. Check all unique values in Geography
2. Create a mapping to standardize country names
3. Use consistent format: "France", "Germany", "Spain"

In [8]:
# Create mapping to standardize country names
# FRA, France, French → France
geography_mapping = {
    'FRA': 'France',
    'France': 'France',
    'French': 'France',
    'Spain': 'Spain',
    'Germany': 'Germany'
}

# Applying standardization
df['Geography'] = df['Geography'].map(geography_mapping)

print("Geography values standardized")

print("="*70)

# Showing standardized values
print("\nAfter standardization:")
print(df['Geography'].value_counts())

Geography values standardized

After standardization:
Geography
France     5014
Germany    2508
Spain      2476
Name: count, dtype: int64


### 2.6 Final Verification of Data Pre-processing

Let's verify all cleaning tasks were completed successfully and the dataset is ready for exploratory analysis.

**Verification Checklist:**
- Missing values handled
- Duplicate columns removed
- All columns have correct data types
- Binary columns properly encoded
- Geography values standardized
- Dataset ready for analysis

In [9]:
print("FINAL DATA QUALITY CHECK")

# 1. Dataset dimensions
print("\n1. Dataset Dimensions:")
print(f"   Rows: {len(df):,}")
print(f"   Columns: {len(df.columns)}")

# 2. Missing values check
print("\n2. Missing Values:")
total_missing = df.isnull().sum().sum()
print(f"   Total missing values: {total_missing}")
if total_missing == 0:
    print("No missing values - dataset is complete")

# 3. Data types verification
print("\n3. Data Types Summary:")
print(f"   Numeric columns: {df.select_dtypes(include=['float64', 'int64']).shape[1]}")
print(f"   Categorical columns: {df.select_dtypes(include=['object']).shape[1]}")

print("\n4. Column Data Types:")
for col in df.columns:
    print(f"   {col:20s}: {str(df[col].dtype):10s}")

# 5. Categorical values check
print("\n5. Categorical Column Values:")
print(f"   Geography: {sorted(df['Geography'].unique())}")
print(f"   Gender: {sorted(df['Gender'].unique())}")

# 6. Binary columns verification
print("\n6. Binary Columns:")
print(f"   HasCrCard values: {sorted(df['HasCrCard'].unique())}")
print(f"   IsActiveMember values: {sorted(df['IsActiveMember'].unique())}")
print(f"   Exited values: {sorted(df['Exited'].unique())}")


print("DATA PRE-PROCESSING COMPLETE")

FINAL DATA QUALITY CHECK

1. Dataset Dimensions:
   Rows: 9,998
   Columns: 13

2. Missing Values:
   Total missing values: 0
No missing values - dataset is complete

3. Data Types Summary:
   Numeric columns: 10
   Categorical columns: 3

4. Column Data Types:
   CustomerId          : int64     
   Surname             : object    
   CreditScore         : int64     
   Geography           : object    
   Gender              : object    
   Age                 : float64   
   Tenure              : int64     
   Balance             : float64   
   NumOfProducts       : int64     
   HasCrCard           : int64     
   IsActiveMember      : int64     
   EstimatedSalary     : float64   
   Exited              : int64     

5. Categorical Column Values:
   Geography: ['France', 'Germany', 'Spain']
   Gender: ['Female', 'Male']

6. Binary Columns:
   HasCrCard values: [np.int64(0), np.int64(1)]
   IsActiveMember values: [np.int64(0), np.int64(1)]
   Exited values: [np.int64(0), np.int64(1)

## 2.7 Exporting Cleaned Dataset

The preprocessed dataset is now ready for analysis. We'll export it as a CSV file for use in the next notebook (Exploratory Data Analysis).

**Export Details:**
- **Format:** CSV (comma-separated values)
- **Filename:** Bank_Churn_Cleaned.csv
- **Location:** C:\Users\HP USER\Documents\Banking-Customer-Churn-Analysis\data
- **Records:** 9,998 customers
- **Columns:** 13 features

In [11]:
# EXPORTING CLEANED DATASET

# Define export path
export_path = 'C:/Users/HP USER/Documents/Banking-Customer-Churn-Analysis/data/Bank_Churn_Cleaned.csv'

df.to_csv(export_path, index=False)    # Export to CSV

print(f"  Dataset exported successfully!")
print(f"  Location: {export_path}")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")

  Dataset exported successfully!
  Location: C:/Users/HP USER/Documents/Banking-Customer-Churn-Analysis/data/Bank_Churn_Cleaned.csv
  Rows: 9,998
  Columns: 13
